In [1]:
import sqlite3
import pandas as pd


In [2]:
conn = sqlite3.connect('sales.db')
cursor = conn.cursor()


In [3]:
cursor.execute("""
CREATE TABLE sales_data (
    Customer_ID INT,
    Customer_Name TEXT,
    Region TEXT,
    Amount INT,
    Order_Date TEXT
)
""")


In [4]:
data = [
    (1, 'Amit', 'South', 5000, '2024-01-02'),
    (2, 'Ravi', 'North', 7000, '2024-01-03'),
    (3, 'Sita', 'South', None, '2024-01-05'),
    (2, 'Ravi', 'North', 7000, '2024-01-03'),
    (4, 'John', 'West', 6500, '2024-01-07')
]

cursor.executemany("INSERT INTO sales_data VALUES (?, ?, ?, ?, ?)", data)
conn.commit()


In [5]:
pd.read_sql_query("SELECT * FROM sales_data WHERE Amount IS NULL", conn)


,Customer_ID,Customer_Name,Region,Amount,Order_Date
0,3,Sita,South,None,2024-01-05


In [6]:
pd.read_sql_query("""
SELECT Customer_ID, COUNT(*)
FROM sales_data
GROUP BY Customer_ID
HAVING COUNT(*) > 1
""", conn)


,Customer_ID,COUNT(*)
0,2,2


In [7]:
pd.read_sql_query("""
SELECT Region, SUM(Amount) AS Total_Sales
FROM sales_data
GROUP BY Region
""", conn)


,Region,Total_Sales
0,North,14000
1,South,5000
2,West,6500


In [8]:
pd.read_sql_query("""
SELECT Order_Date, SUM(Amount) AS Daily_Sales
FROM sales_data
GROUP BY Order_Date
""", conn)


,Order_Date,Daily_Sales
0,2024-01-02,5000.0
1,2024-01-03,14000.0
2,2024-01-05,NaN
3,2024-01-07,6500.0


In [9]:
report = pd.read_sql_query("""
SELECT Region, SUM(Amount) AS Total_Sales
FROM sales_data
GROUP BY Region
""", conn)

report.to_csv("sql_sales_report.csv", index=False)
